In [1]:
import cv2
import numpy as np

def identify_piece(square_img, templates):
    square_gray = cv2.cvtColor(square_img, cv2.COLOR_BGR2GRAY)
    best_match, best_score = None, 0.7  # Adjust threshold as needed
    for piece, tmpl in templates.items():
        res = cv2.matchTemplate(square_gray, tmpl, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(res)
        if max_val > best_score:
            best_score = max_val
            best_match = piece
    return best_match

In [2]:
import pyautogui

# Define the region to capture (left, top, width, height)
board_region = (100, 100, 480, 480)  # Adjust these values as needed

# Capture the screenshot
screenshot = pyautogui.screenshot(region=board_region)
screenshot.save("board.png")

In [36]:
import cv2
import os

def load_templates(template_folder='assets'):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith('.png'):
            piece = filename.replace('.png', '')
            print(f"Loading template for {piece} from {filename}")
            path = os.path.join(template_folder, filename)
            templates[piece] = cv2.imread(path)  # Load as grayscale
    return templates

templates = load_templates()

Loading template for bB from bB.png
Loading template for bK from bK.png
Loading template for bN from bN.png
Loading template for bP from bP.png
Loading template for bQ from bQ.png
Loading template for bR from bR.png
Loading template for wB from wB.png
Loading template for wK from wK.png
Loading template for wN from wN.png
Loading template for wP from wP.png
Loading template for wQ from wQ.png
Loading template for wR from wR.png


In [ ]:
# why do my templates have same values?
# I think I need to convert them to grayscale
# and then use the same method to match them
# to the screenshot


import cv2
import os

def load_templates(template_folder='assets'):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith('.png'):
            piece = filename.replace('.png', '')
            print(f"Loading template for {piece} from {filename}")
            path = os.path.join(template_folder, filename)
            templates[piece] = cv2.imread(path)  # Load as grayscale
    return templates

templates = load_templates()



Loading template for bB from bB.png
Loading template for bK from bK.png
Loading template for bN from bN.png
Loading template for bP from bP.png
Loading template for bQ from bQ.png
Loading template for bR from bR.png
Loading template for wB from wB.png
Loading template for wK from wK.png
Loading template for wN from wN.png
Loading template for wP from wP.png
Loading template for wQ from wQ.png
Loading template for wR from wR.png


In [9]:
import numpy as np

def identify_piece(square_img, templates):
    square_gray = cv2.cvtColor(square_img, cv2.COLOR_BGR2GRAY)
    best_match, best_score = None, 0.7  # Adjust threshold as needed
    for piece, tmpl in templates.items():
        res = cv2.matchTemplate(square_gray, tmpl, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(res)
        if max_val > best_score:
            best_score = max_val
            best_match = piece
    return best_match

def build_board_from_image(board_img, templates):
    board = [['' for _ in range(8)] for _ in range(8)]
    square_size = board_img.shape[0] // 8

    for row in range(8):
        for col in range(8):
            x1 = col * square_size
            y1 = row * square_size
            square_img = board_img[y1:y1+square_size, x1:x1+square_size]
            piece = identify_piece(square_img, templates)
            board[row][col] = piece if piece else ''
    return board

In [10]:
def board_to_fen(board):
    rows = []
    for row in board:
        fen_row = ''
        empty = 0
        for piece in row:
            if not piece:
                empty += 1
            else:
                if empty:
                    fen_row += str(empty)
                    empty = 0
                fen_row += piece[1].upper() if piece[0] == 'w' else piece[1].lower()
        if empty:
            fen_row += str(empty)
        rows.append(fen_row)
    return '/'.join(rows) + ' w KQkq - 0 1'

In [11]:
import chess
import chess.engine

def evaluate_fen(fen, stockfish_path='stockfish'):
    board = chess.Board(fen)
    engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
    result = engine.analyse(board, chess.engine.Limit(time=2.0))
    print("Evaluation:", result["score"])
    engine.quit()

In [ ]:
import time
import pyautogui
import cv2
import numpy as np

def main():
    templates = load_templates()
    while True:
        screenshot = pyautogui.screenshot(region=board_region)
        board_img = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
        board_grid = build_board_from_image(board_img, templates)
        fen = board_to_fen(board_grid)
        evaluate_fen(fen)
        time.sleep(5)  # Adjust the interval as needed

if __name__ == '__main__':
    main()

In [47]:
import time
import pyautogui
import cv2
import numpy as np
import chess
import chess.engine
import os

# Path to Stockfish executable
STOCKFISH_PATH = r'D:\Sharan\College\Kaggle Competitions\Self-Projects\Chess Com\stockfish\stockfish-windows-x86-64-avx2.exe'

# Path to folder where piece templates are stored
TEMPLATE_FOLDER = 'assets'

# Template for an empty light square to detect the board
SQUARE_TEMPLATE_PATH = 'board_temp.png'  # Use a light square template for detection

# Load piece templates (wP, bK, etc.)
def load_templates(template_folder='assets_new'):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith('.png'):
            piece = filename.replace('.png', '')
            path = os.path.join(template_folder, filename)
            templates[piece] = cv2.imread(path)  # Load as colored image (no grayscale conversion)
    return templates

# Capture full screenshot
def capture_fullscreen():
    screenshot = pyautogui.screenshot()
    img = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
    return img

# Search for the board in the screen
# Search for the board in the screen (colored image matching)
def find_board(image, square_template):
    # Ensure the image and template are both the same type and depth (CV_8U or CV_32F)
    
    # If the input image is colored and template is colored, ensure they both have the same depth
    if image.dtype != square_template.dtype:
        # Convert the image and template to the same type (CV_8U)
        image = np.array(image, dtype=np.uint8)
        square_template = np.array(square_template, dtype=np.uint8)
    
    # Perform template matching using colored images (no grayscale conversion)
    result = cv2.matchTemplate(image, square_template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)

    if max_val < 0.8:  # Confidence threshold
        return None

    board_height, board_width = square_template.shape[:2]
    # Return the position of the detected board region
    x, y = max_loc
    # square_size = square_template.shape[0]
    # board_size = square_size * 8
    return (x, y, board_width, board_height)  # Return board region

# Identify a piece on a given square
def identify_piece(square_img, templates):
    square_gray = cv2.cvtColor(square_img, cv2.COLOR_BGR2GRAY)
    best_match, best_score = None, 0.7  # Adjust threshold as needed
    for piece, tmpl in templates.items():
        res = cv2.matchTemplate(square_gray, tmpl, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(res)
        if max_val > best_score:
            best_score = max_val
            best_match = piece
    return best_match

# Build the chessboard and return a list of piece positions
def build_board_from_image(board_img, templates):
    board = [['' for _ in range(8)] for _ in range(8)]
    square_size = board_img.shape[0] // 8  # Assume square board with 8x8 grid

    for row in range(8):
        for col in range(8):
            x1 = col * square_size
            y1 = row * square_size
            square_img = board_img[y1:y1+square_size, x1:x1+square_size]
            piece = identify_piece(square_img, templates)
            board[row][col] = piece if piece else ''
    return board

# Convert board to FEN string
def board_to_fen(board):
    rows = []
    for row in board:
        fen_row = ''
        empty = 0
        for piece in row:
            if not piece:
                empty += 1
            else:
                if empty:
                    fen_row += str(empty)
                    empty = 0
                fen_row += piece[1].upper() if piece[0] == 'w' else piece[1].lower()
        if empty:
            fen_row += str(empty)
        rows.append(fen_row)
    return '/'.join(rows) + ' w KQkq - 0 1'

# Evaluate position using Stockfish
def evaluate_fen(fen, stockfish_path=STOCKFISH_PATH):
    board = chess.Board(fen)
    engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
    result = engine.analyse(board, chess.engine.Limit(time=2.0))  # 2 second analysis
    print("Evaluation:", result["score"])
    engine.quit()

# Main loop
def main():
    templates = load_templates()
    square_template = cv2.imread(SQUARE_TEMPLATE_PATH)  # Read light square template
    # print(square_template.shape)
    while True:
        full_img = capture_fullscreen()
        board_region = find_board(full_img, square_template)

        if board_region is None:
            # If board is not found, keep trying
            print("Board not found. Retrying...")
            time.sleep(2)
            continue

        # # If board is found, crop the region to match the board
        x, y, w, h = board_region

        # cropped_board = full_img[y:y + w, x:x + h]
        board_img = full_img[y:y+h, x:x+w]

        # Save board image for debugging
        cv2.imwrite('board.png', board_img)
        
        # Process the board image
        board_grid = build_board_from_image(board_img, templates)

        # # Generate FEN and evaluate
        # fen = board_to_fen(board_grid)
        # print("Generated FEN:", fen)
        # evaluate_fen(fen)

        # # Sleep for 5 seconds before the next screenshot
        # time.sleep(5)

if __name__ == '__main__':
    main()


Board not found. Retrying...
Board not found. Retrying...
Board not found. Retrying...


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\templmatch.cpp:1164: error: (-215:Assertion failed) (depth == CV_8U || depth == CV_32F) && type == _templ.type() && _img.dims() <= 2 in function 'cv::matchTemplate'


In [53]:
import time
import pyautogui
import cv2
import numpy as np
import chess
import chess.engine
import os

# Path to Stockfish executable
STOCKFISH_PATH = r'D:\Sharan\College\Kaggle Competitions\Self-Projects\Chess Com\stockfish\stockfish-windows-x86-64-avx2.exe'

# Path to folder where piece templates are stored
TEMPLATE_FOLDER = 'assets'

# Template for an empty light square to detect the board
SQUARE_TEMPLATE_PATH = 'board_temp.png'  # Use a light square template for detection

# Load piece templates (wP, bK, etc.)
def load_templates(template_folder='assets'):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith('.png'):
            piece = filename.replace('.png', '')
            path = os.path.join(template_folder, filename)
            templates[piece] = cv2.imread(path)  # Load as colored image (no grayscale conversion)
    return templates

# Capture full screenshot
def capture_fullscreen():
    screenshot = pyautogui.screenshot()
    img = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
    return img

# Search for the board in the screen (colored image matching)
def find_board(image, square_template):
    # Ensure the image and template are both the same type and depth (CV_8U or CV_32F)
    if image.dtype != square_template.dtype:
        image = np.array(image, dtype=np.uint8)
        square_template = np.array(square_template, dtype=np.uint8)
    
    # Perform template matching using colored images (no grayscale conversion)
    result = cv2.matchTemplate(image, square_template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)

    if max_val < 0.8:  # Confidence threshold
        return None

    board_height, board_width = square_template.shape[:2]
    # Return the position of the detected board region
    x, y = max_loc
    return (x, y, board_width, board_height)  # Return board region

# Identify a piece on a given square
def identify_piece(square_img, templates):
    best_match, best_score = None, 0.7  # Adjust threshold as needed

    for piece, tmpl in templates.items():
        res = cv2.matchTemplate(square_img, tmpl, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(res)
        if max_val > best_score:
            best_score = max_val
            best_match = piece

    return best_match

# Build the chessboard and return a list of piece positions
def build_board_from_image(board_img, templates):
    board = [['' for _ in range(8)] for _ in range(8)]
    square_size = board_img.shape[0] // 8  # Assume square board with 8x8 grid

    for row in range(8):
        for col in range(8):
            x1 = col * square_size
            y1 = row * square_size
            square_img = board_img[y1:y1 + square_size, x1:x1 + square_size]
            piece = identify_piece(square_img, templates)
            board[row][col] = piece if piece else ''
    return board

# Convert board to FEN string
def board_to_fen(board):
    rows = []
    for row in board:
        fen_row = ''
        empty = 0
        for piece in row:
            if not piece:
                empty += 1
            else:
                if empty:
                    fen_row += str(empty)
                    empty = 0
                fen_row += piece[1].upper() if piece[0] == 'w' else piece[1].lower()
        if empty:
            fen_row += str(empty)
        rows.append(fen_row)
    return '/'.join(rows) + ' w KQkq - 0 1'

# Evaluate position using Stockfish
def evaluate_fen(fen, stockfish_path=STOCKFISH_PATH):
    board = chess.Board(fen)
    engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
    result = engine.analyse(board, chess.engine.Limit(time=2.0))  # 2 second analysis
    print("Evaluation:", result["score"])
    engine.quit()

# Main loop
def main():
    templates = load_templates()  # Load the piece templates
    square_template = cv2.imread(SQUARE_TEMPLATE_PATH)  # Read light square template
    while True:
        full_img = capture_fullscreen()  # Capture full screenshot
        board_region = find_board(full_img, square_template)

        if board_region is None:
            # If board is not found, keep trying
            print("Board not found. Retrying...")
            time.sleep(2)
            continue

        # If board is found, crop the region to match the board
        x, y, w, h = board_region
        board_img = full_img[y:y+h, x:x+w]  # Crop the board region

        # Process the board image to get piece positions
        board_grid = build_board_from_image(board_img, templates)

        # Generate FEN and evaluate position
        fen = board_to_fen(board_grid)
        print("Generated FEN:", fen)
        evaluate_fen(fen)

        # Sleep for a while before the next screenshot
        time.sleep(5)

if __name__ == '__main__':
    main()


Board not found. Retrying...
Generated FEN: 1BBBBBBB/Kbbbbbbb/Kbbbbbbb/Kbbbbbbb/Kbbbbbbb/Kbbbbbbb/Kbbbbbbb/Kbbbbbbb w KQkq - 0 1


NotImplementedError: 